# Read CSV

In [1]:
import csv
from functools import reduce

def read_csv(file_name):
    with open(file_name, "r") as f:
        pcb_names = list(csv.reader(f, delimiter=","))

    pcb_names = reduce(lambda xs, ys: xs + ys, pcb_names)
    return pcb_names

# Deal with fp_text

In [1]:
from collections import defaultdict

def revise_module_fp_text(pcb):

    text_count = defaultdict(int)
    for m in pcb.module:
        text_count[m.fp_text[0][1]] += 1

    print(text_count)
    for m in pcb.module:
        if text_count[m.fp_text[0][1]] > 1 or m.fp_text[0][1] == '""' or m.fp_text[0][1] == '" "':
            tmp_text = m.fp_text[0][1]
            if tmp_text == '""':
                m.fp_text[0][1] =  f"Noname_{text_count[tmp_text]}"
            elif tmp_text == '" "':
                m.fp_text[0][1] =  f"None_{text_count[tmp_text]}"
            else:
                m.fp_text[0][1] =  str(tmp_text) + "_" + str(text_count[tmp_text])
            del m.fp_text[0][1][0]
            text_count[tmp_text] -= 1

# Deal with fill zones

In [3]:
import sys
sys.path.append('..')
from Data_extraction.thirdparty.kicad_parser.kicad_pcb import *

import os

def delete_segment_via(pcb, delete_nets):

    seg_idx = []
    via_idx = []
    for i in range(len(pcb.segment)):
        if pcb.segment[i].net in delete_nets:
            seg_idx.append(i)
    seg_idx.sort()
    for i in range(len(pcb.via)):
        if pcb.via[i].net in delete_nets:
            via_idx.append(i)
    via_idx.sort()
    # print(seg_idx, via_idx)

    bc = 0
    for i in seg_idx:
        del_i = i - bc
        if pcb.segment[del_i].net in delete_nets:
            del pcb.segment[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for segment")

    bc = 0
    for i in via_idx:
        del_i = i - bc
        if pcb.via[del_i].net in delete_nets:
            del pcb.via[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for via")


pcb_names = read_csv("route_fill_zones.csv")

print(len(pcb_names), pcb_names)

fill_zone_folder = "deleted_fill_zones"
if not os.path.exists(fill_zone_folder):
    os.mkdir(fill_zone_folder)

for name in pcb_names:
    pcb_file_path = "../../PCBs_new/" + name + "/raw.kicad_pcb"
    new_pcb_name = name + ".kicad_pcb"
    pcb = KicadPCB.load(pcb_file_path)
    fill_zone_nets = set([])
    for zone in pcb.zone:
        fill_zone_nets.add(zone.net)
    print(name)
    del pcb.zone
    delete_segment_via(pcb, fill_zone_nets)
    revise_module_fp_text(pcb)
    pcb.export(os.path.join(fill_zone_folder, new_pcb_name))



80 ['bishboria_ErgoDox', 'cnlohr_wiflier_B', '1Bitsy_1bitsy', 'cnlohr_wiflier_EXP', 'cnlohr_wiflier_EXP_B', 'devttys0_IRis', 'hamityanik_ULP-Weather-Logger', 'hwstar_ac-power-monitor', 'jadonk_PocketBone', 'JavierIH_kameshield', 'kitspace__autosave-nunchuk_breakout', 'kitspace__autosave-postcard', 'kitspace_4_switch_array', 'kitspace_8_switch_array', 'kitspace_12_24_boost_converter', 'kitspace_12V5A_breakout', 'kitspace_40-channel-hv-switching-board', 'kitspace_120-channel-pogo-pin-board', 'kitspace_120-channel-test-board', 'kitspace_aquarius', 'kitspace_ardfpga', 'kitspace_Arduino_uno_sheild', 'kitspace_BalthazarKeyboard3-keycaps', 'kitspace_bomanz', 'kitspace_Control%20Board', 'kitspace_d20_r0.1', 'kitspace_d20_tri_r0.2', 'kitspace_d20_tri_r0.3', 'kitspace_Driverino-Shield', 'kitspace_dropbot_control_board', 'kitspace_dropbot-front-panel', 'kitspace_EEZ%20DIB%20MCU%20r1B2', 'kitspace_ESPTINY86_Mixtape_version2', 'kitspace_esptiny86_Stompbox', 'kitspace_firefly', 'kitspace_flypi', 'ki

# Check FreeRouting results

In [2]:
import os

pcb_names = read_csv("route_fill_zones.csv")
results_folder = "./deleted_fill_zones/"
resuts_files = list(os.listdir(results_folder))
for pcb_name in pcb_names:
    res_name = pcb_name + ".ses"
    if res_name not in resuts_files:
        print(pcb_name)
    

kitspace_d20_r0.1
kitspace_d20_tri_r0.2
kitspace_d20_tri_r0.3
kitspace_EEZ%20DIB%20MCU%20r1B2
kitspace_glasgow
kitspace_lpa96
kitspace_mppt-1210-hus
kitspace_OtterCastAmp
kitspace_tri_panel_4up_r0.1
kitspace_ulx3s
kitspace_ulx4m
kitspace_USB-C-Screen-Adapter
LibreSolar_MPPT_charger_20A


# Update PCBs folder

Put processed.kicad_pcb into folder

In [3]:
import os
import shutil

good_raw = read_csv("./good_raw.csv")
fill_zones_pcbs = read_csv("./route_fill_zones.csv")
print(good_raw)
print(fill_zones_pcbs)

pcb_folder = '../../PCBs/'
for pcb in os.listdir(pcb_folder):
    target = '../../PCBs/' + pcb + '/processed.kicad_pcb'
    if pcb in good_raw:
        source = '../../PCBs_new/' + pcb + '/raw.kicad_pcb'
        shutil.copy(source, target)
    elif pcb in fill_zones_pcbs:
        source = './deleted_fill_zones/' + pcb + ".kicad_pcb"
        shutil.copy(source, target)
    elif os.path.exists(target):
        os.remove(target)

    if os.path.exists('../../PCBs/' + pcb + '/error.log'):
        os.remove('../../PCBs/' + pcb + '/error.log')

    if os.path.exists('../../PCBs/' + pcb + '/final.json'):
        os.remove('../../PCBs/' + pcb + '/final.json')


['kitspace_antenna_analyser', 'bobc_control_panel', 'bobc_matrix_clock', 'bobc_LCD-panel-adapter-lvc', 'bobc_mbeduinopresso', 'bobc_MS-F100', 'cnlohr_wiflier', 'kitspace_bbq10breakout', 'kitspace_BQ25570_Harvester', 'kitspace_Brk40p', 'kitspace_CA3306E', 'kitspace_CH330', 'kitspace_ChaosLooper', 'kitspace_CO2', 'kitspace_CocoMixtape_UGM_Kicad', 'kitspace_cseduinov4', 'kitspace_DIY_detector', 'kitspace_dynamixel_shield', 'kitspace_esp8266', 'kitspace_esp8266-12-breakout', 'kitspace_ESP8266-PowerMonitor', 'kitspace_f-91w', 'kitspace_fast_diode_pcb', 'kitspace_filaSens', 'kitspace_gas_sensor', 'kitspace_GPSMux', 'kitspace_grove_adaptor', 'kitspace_hbridge_driver', 'kitspace_Heltec_Node', 'kitspace_hum_temp_sensor', 'kitspace_ir_sensor', 'kitspace_Mixtape_Barebone', 'kitspace_nunchuk_breakout', 'kitspace_OutlineChaos', 'kitspace_PCBs', 'kitspace_pmt_combiner', 'kitspace_postcard', 'kitspace_Potentiometer_mount_4LED', 'kitspace_Potentiometer_mount_8LED', 'kitspace_Potentiometer_mount_16LED'

In [16]:
import sys
sys.path.append('..')
from Data_extraction.thirdparty.kicad_parser.kicad_pcb import *
pcb = KicadPCB.load("../../PCBs/1Bitsy_1bitsy/raw.kicad_pcb")
for m in pcb.module:
    if m.fp_text[0][1] == "REF**":
        a = m.fp_text[0][1]
        m.fp_text[0][1] = "test"
    print(m.fp_text[0][1])

<list>:2
<list>:2
<list>:2
J1
<list>:2
<list>:2
C1
C2
C3
C4
C5
C8
C9
C10
C11
C12
C13
C14
C15
C16
C17
C18
D1
L1
LED1
P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12
P13
P14
P15
P16
P17
P18
P19
P20
P21
P22
P23
P24
P25
P26
P27
P28
P29
P30
P31
P32
P33
P34
P35
P36
P37
P38
P39
R1
R2
R3
R4
R5
U1
U2
X2
X3
P40
P41
SW1
C6
C7
P42
P53
X1
P43
P44
P45
P46
P47
P48
P49
P50
P51
P52
P54
P55
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
<list>:2
